# | default_exp _code_generator.asyncapi_spec_generator

In [ ]:
# | export

from typing import *
import time
import platform
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.
from pathlib import Path
from tempfile import TemporaryDirectory

from yaspin import yaspin
import yaml
from packaging import version

from fastkafka._components.docs_dependencies import _check_npm_with_local, npm_required_major_version

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import CustomAIChat, ValidateAndFixResponse, write_file_contents
from faststream_gen._code_generator.prompts import ASYNCAPI_SPEC_GENERATION_PROMPT
from faststream_gen._code_generator.constants import ASYNC_API_SPEC_FILE_NAME, MAX_ASYNC_SPEC_RETRIES, MAX_NUM_FIXES_MSG

23-09-06 10:58:45.037 [INFO] numexpr.utils: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
23-09-06 10:58:45.038 [INFO] numexpr.utils: NumExpr defaulting to 8 threads.


In [ ]:
import shutil
import pytest
import os
from contextlib import contextmanager

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _extract_errors(output: str, spec_dir: str) -> List[str]:
    """Extract error messages from the AsyncAPI CLI validation output.

    Args:
        output: The output of the AsyncAPI CLI validation command.
        base_directory (str): The base directory name of the generated AsyncAPI spec file.

    Returns:
        A list of error messages generated by AsyncAPI CLI validation command.
    """
    output_lines = output.split("\n")
    errors = [
        line.replace(f"{spec_dir}/", "") for line in output_lines if " error " in line
    ]
    return errors

In [ ]:
fixture_output = """File fastkafka-gen/asyncapi.yml and/or referenced documents have governance issues.
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi-defaultContentType "AsyncAPI document should have "defaultContentType" field."
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi-id "AsyncAPI document should have "id" field."
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi2-tags "AsyncAPI object should have non-empty "tags" array."
fastkafka-gen/asyncapi.yml:1:11 information asyncapi-latest-version "The latest version of AsyncAPi is not used. It is recommended update to the "2.6.0" version."
fastkafka-gen/asyncapi.yml:2:6 warning asyncapi-info-license "Info object should have "license" object."
fastkafka-gen/asyncapi.yml:38:15 warning asyncapi2-operation-operationId "Operation should have an "operationId" field defined."
fastkafka-gen/asyncapi.yml:43:13 warning asyncapi2-operation-operationId "Operation should have an "operationId" field defined."
fastkafka-gen/asyncapi.yml:49:18 error asyncapi-document-resolved ""message" property must have required property "oneOf""
fastkafka-gen/asyncapi.yml:49:18 error asyncapi-document-resolved ""StoreProduct" property must have required property "oneOf""
fastkafka-gen/asyncapi.yml:49:18 warning asyncapi2-message-messageId "Message should have a "messageId" field defined."
fastkafka-gen/asyncapi.yml:50:15 error asyncapi-document-resolved "Property "payload" is not expected to be here"
fastkafka-gen/asyncapi.yml:65:14 error asyncapi-document-resolved "Property "title" is not expected to be here"
fastkafka-gen/asyncapi.yml:66:13 error asyncapi-document-resolved "Property "type" is not expected to be here"
"""
expected = [
    'asyncapi.yml:49:18 error asyncapi-document-resolved ""message" property must have required property "oneOf""',
    'asyncapi.yml:49:18 error asyncapi-document-resolved ""StoreProduct" property must have required property "oneOf""',
    'asyncapi.yml:50:15 error asyncapi-document-resolved "Property "payload" is not expected to be here"',
    'asyncapi.yml:65:14 error asyncapi-document-resolved "Property "title" is not expected to be here"',
    'asyncapi.yml:66:13 error asyncapi-document-resolved "Property "type" is not expected to be here"',
]


spec_path = "fastkafka-gen"
actual = _extract_errors(fixture_output, spec_path)
print(actual)
assert actual == expected

['asyncapi.yml:49:18 error asyncapi-document-resolved ""message" property must have required property "oneOf""', 'asyncapi.yml:49:18 error asyncapi-document-resolved ""StoreProduct" property must have required property "oneOf""', 'asyncapi.yml:50:15 error asyncapi-document-resolved "Property "payload" is not expected to be here"', 'asyncapi.yml:65:14 error asyncapi-document-resolved "Property "title" is not expected to be here"', 'asyncapi.yml:66:13 error asyncapi-document-resolved "Property "type" is not expected to be here"']


In [ ]:
fixture_output = """File fastkafka-gen/asyncapi.yml and/or referenced documents have governance issues.
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi-defaultContentType "AsyncAPI document should have "defaultContentType" field."
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi-id "AsyncAPI document should have "id" field."
fastkafka-gen/asyncapi.yml:1:1 warning asyncapi2-tags "AsyncAPI object should have non-empty "tags" array."
fastkafka-gen/asyncapi.yml:1:11 information asyncapi-latest-version "The latest version of AsyncAPi is not used. It is recommended update to the "2.6.0" version."
fastkafka-gen/asyncapi.yml:2:6 warning asyncapi-info-license "Info object should have "license" object."
fastkafka-gen/asyncapi.yml:38:15 warning asyncapi2-operation-operationId "Operation should have an "operationId" field defined."
fastkafka-gen/asyncapi.yml:43:13 warning asyncapi2-operation-operationId "Operation should have an "operationId" field defined."
fastkafka-gen/asyncapi.yml:49:18 warning asyncapi2-message-messageId "Message should have a "messageId" field defined."
"""
expected = []


spec_path = "fastkafka-gen"
actual = _extract_errors(fixture_output, spec_path)
print(actual)
assert actual == expected

[]


In [ ]:
npm_path = "/".join(shutil.which("npm").split("/")[:-1])

@contextmanager
def _remove_npm_from_path():
    try:
        original_path = os.environ["PATH"]
        os.environ["PATH"] = original_path.replace(f"{npm_path}:", "").replace(":/bin:", ":")
        yield
    finally:
        os.environ["PATH"] = original_path
        
        

with _remove_npm_from_path():
    actual = os.environ["PATH"]
#     print(actual)
    assert npm_path not in actual
    assert ":/bin:" not in actual

actual = os.environ["PATH"]
# print(actual)
assert npm_path in actual
print("ok")

ok


In [ ]:
# | export


def _validate_response(response: str) -> List[str]:
    """Validate the AsyncAPI spec generated by OpenAI

    Args:
        response: The AsyncAPI spec generated by OpenAI in string format.

    Returns:
        Returns a list of errors if any found during the validation of the spec.

    Raises:
        json.JSONDecodeError: If the response is not a valid JSON.
    """    
    # check if nmp is installed
    try:
        _check_npm_with_local()
    except RuntimeError as e:
        raise RuntimeError(
            f"Error: npm not found. To use the code generation feature, you must have npm >= {npm_required_major_version} installed.\nPlease run the following command to install the required dependencies:\n\nfastkafka docs install_deps"
        )

    with TemporaryDirectory() as d:
        spec_path = Path(d) / ASYNC_API_SPEC_FILE_NAME
        with open(spec_path, "w", encoding="utf-8") as f:
            f.write(response)

        cmd = [
            "npx",
            "-y",
            "-p",
            "@asyncapi/cli",
            "asyncapi",
            "validate",
            f"{spec_path}",
            "--diagnostics-format",
            "text",
        ]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.STDOUT,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        if p.returncode == 0:
            errors = _extract_errors(p.stdout.decode(), str(d))
            return errors
        else:
            logger.info(f"Validation of AsyncAPI spec failed!")
            logger.info(f"Output of '$ {' '.join(cmd)}'{p.stdout.decode()}")
            raise ValueError(
                f"Validation of AsyncAPI spec failed, '$ {' '.join(cmd)}'{p.stdout.decode()}.\n\nPlease try again."
            )

In [ ]:
invalid_yaml = """
grandparent:
  parent:
    child:
      name: Bobby
    sibling:
      name: Molly
"""
with _remove_npm_from_path():
    with pytest.raises(RuntimeError) as e:
        _validate_response(invalid_yaml)

print(e.value)

Error: npm not found. To use the code generation feature, you must have npm >= 9 installed.
Please run the following command to install the required dependencies:

fastkafka docs install_deps


In [ ]:
invalid_yaml = """
grandparent:
  parent:
    child:
      name: Bobby
    sibling:
      name: Molly
"""

actual = _validate_response(invalid_yaml)
print(actual)
expected = ['asyncapi.yml:1:1 error asyncapi-is-asyncapi "This is not an AsyncAPI document. The "asyncapi" field as string is missing."']
assert actual == expected

['asyncapi.yml:1:1 error asyncapi-is-asyncapi "This is not an AsyncAPI document. The "asyncapi" field as string is missing."']


In [ ]:
invalid_yaml = """asyncapi: 2.5.0
info:
  title: Currency Conversion
  version: 0.0.1
  description: "A FastKafka application which consumes JSON-encoded messages from the 'store_product' topic. It checks if the currency attribute is set to 'HRK' and converts the currency to 'EUR' by dividing the price by 7.5. The converted message is then published to the 'change_currency' topic. The application utilizes a localhost broker for testing, staging.airt.ai for staging, and prod.airt.ai for production. It uses SASL_SSL with SCRAM-SHA-256 for authentication."
  contact:
    name: Author
    url: https://www.google.com/
    email: noreply@gmail.com
servers:
  localhost:
    url: localhost
    description: local development kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
  staging:
    url: staging.airt.ai
    description: staging kafka broker
    protocol: kafka-secure
    security:
    - staging_default_security: []
    variables:
      port:
        default: '9092'
  production:
    url: prod.airt.ai
    description: production kafka broker
    protocol: kafka-secure
    security:
    - production_default_security: []
    variables:
      port:
        default: '9092'
channels:
  store_product:
    subscribe:
      message:
        $ref: '#/components/messages/StoreProduct'
      description: "For each consumed message, the application checks if the currency attribute is set to 'HRK' and converts the currency to 'EUR' by dividing the price by 7.5. The modified message is then published to the 'change_currency' topic."
  change_currency:
    publish:
      message:
        $ref: '#/components/messages/StoreProduct'
      description: "Publishes the consumed message to the 'change_currency' topic."
components:
  messages:
    StoreProduct:
      payload:
        properties:
          product_name:
            type: string
            description: Name of the product
          currency:
            type: string
            description: Currency (three letter string)
          price:
            type: number
            description: Price of the product
        required:
          - product_name
          - currency
          - price
      title: Store Product
      type: object
  schemas: {}
  securitySchemes:
    staging_default_security:
      type: scramSha256
    production_default_security:
      type: scramSha256
"""

actual = _validate_response(invalid_yaml)
print(actual)
expected = [
    'asyncapi.yml:49:18 error asyncapi-document-resolved ""message" property must have required property "oneOf""',
    'asyncapi.yml:49:18 error asyncapi-document-resolved ""StoreProduct" property must have required property "oneOf""',
    'asyncapi.yml:50:15 error asyncapi-document-resolved "Property "payload" is not expected to be here"',
    'asyncapi.yml:65:14 error asyncapi-document-resolved "Property "title" is not expected to be here"',
    'asyncapi.yml:66:13 error asyncapi-document-resolved "Property "type" is not expected to be here"',
]
assert actual == expected

['asyncapi.yml:49:18 error asyncapi-document-resolved ""message" property must have required property "oneOf""', 'asyncapi.yml:49:18 error asyncapi-document-resolved ""StoreProduct" property must have required property "oneOf""', 'asyncapi.yml:50:15 error asyncapi-document-resolved "Property "payload" is not expected to be here"', 'asyncapi.yml:65:14 error asyncapi-document-resolved "Property "title" is not expected to be here"', 'asyncapi.yml:66:13 error asyncapi-document-resolved "Property "type" is not expected to be here"']


In [ ]:
valid_yaml = """asyncapi: 2.5.0
info:
  title: Greet users
  version: 0.0.1
  description: 'Create a FastKafka application using localhost broker for testing,
    staging.airt.ai for staging and prod.airt.ai for production. Use default port
    number. It should consume messages from ''receive_name'' topic and the message
    will be a JSON encoded object with only one attribute: user_name. For each consumed
    message, construct a new message object and append ''Hello '' in front of the
    name attribute. Finally, publish the consumed message to ''send_greetings'' topic.'
  contact:
    name: Author
    url: https://www.google.com/
    email: noreply@gmail.com
servers:
  localhost:
    url: localhost
    description: local development kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
  staging:
    url: staging.airt.ai
    description: staging kafka broker
    protocol: kafka-secure
    security:
    - staging_default_security: []
    variables:
      port:
        default: '9092'
  production:
    url: prod.airt.ai
    description: production kafka broker
    protocol: kafka-secure
    security:
    - production_default_security: []
    variables:
      port:
        default: '9092'
channels:
  receive_name:
    subscribe:
      message:
        $ref: '#/components/messages/Greetings'
      description: For each consumed message, construct a new message object and append
        'Hello ' in front of the name attribute. Finally, publish the consumed message
        to 'send_greetings' topic.
  send_greetings:
    publish:
      message:
        $ref: '#/components/messages/Greetings'
components:
  messages:
    Greetings:
      payload:
        properties:
          user_name:
            description: Name of the user.
            title: User Name
            type: string
        required:
        - user_name
        title: Greetings
        type: object
  schemas: {}
  securitySchemes:
    staging_default_security:
      type: scramSha256
    production_default_security:
      type: scramSha256
"""

actual = _validate_response(valid_yaml)
print(actual)
expected = []
assert actual == expected

[]


In [ ]:
# | export

def _asyncapi_to_latest_version(asyncapi_yaml_path: str) -> None:
    """
    Convert the AsyncAPI specification to the latest version.

    Args:
        asyncapi_yaml_path: The AsyncAPI specification which needs to be converted.
    """
    with open(asyncapi_yaml_path, "r") as stream:
        current_version = yaml.safe_load(stream)["asyncapi"]
        
    LATEST_ASYNCAPI_VERSION = "2.6.0"
    if version.parse(current_version) < version.parse(LATEST_ASYNCAPI_VERSION):
        cmd = [
            "npx",
            "-y",
            "-p",
            "@asyncapi/cli",
            "asyncapi",
            "convert",
            f"{asyncapi_yaml_path}",
            "-t",
            f"{LATEST_ASYNCAPI_VERSION}",
            "-o",
            f"{asyncapi_yaml_path}",
        ]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.STDOUT,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        
        logger.info("Executing 'asyncapi convert' on the generated asyncapi specification file:")
        if p.returncode == 0:
            current_version = LATEST_ASYNCAPI_VERSION
        else:   
            logger.info(f"Issues while executing 'asyncapi convert' command: {p.stdout.decode()}")
            
    logger.info(f"Using AsyncAPI version: {current_version} for the specification creation")

In [ ]:
with TemporaryDirectory() as d:
    spec_path = Path(d) / ASYNC_API_SPEC_FILE_NAME
    with open(spec_path, "w", encoding="utf-8") as f:
        f.write(valid_yaml)
        
    _asyncapi_to_latest_version(spec_path)

[INFO] __main__: Executing 'asyncapi convert' on the generated asyncapi specification file:
[INFO] __main__: Using AsyncAPI version: 2.6.0 for the specification creation


In [ ]:
# | export

def _optimize_asyncapi_file(asyncapi_yaml_path: str) -> None:
    """
    Optimize the AsyncAPI specificationn.

    Args:
        asyncapi_yaml_path: The AsyncAPI specification which needs to be optimized.
    """
    cmd = [
        "npx",
        "-y",
        "-p",
        "@asyncapi/cli",
        "asyncapi",
        "optimize",
        f"{asyncapi_yaml_path}",
        "-o",
        "overwrite",
    ]
    # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
    p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
        cmd,
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE,
        shell=True if platform.system() == "Windows" else False,
    )

    # Note: asyncapi cli is returning incorrect log. 
    # If we want to optimize asyncapi.yml, we will get the message "Created file asyncapi_optimized.yml"
    # asyncapi_optimized.yml ISN'T created - asyncapi.yml was overwritten (we are using attribute -o overwrite)
 
    logger.info("Executing 'asyncapi optimize' on the generated asyncapi specification file:")
    if p.returncode == 0:
        logger.info(p.stdout.decode())
    else:   
        logger.info(f"Issues while executing 'asyncapi optimize' command: {p.stdout.decode()}")

In [ ]:
with TemporaryDirectory() as d:
    spec_path = Path(d) / ASYNC_API_SPEC_FILE_NAME
    with open(spec_path, "w", encoding="utf-8") as f:
        f.write(valid_yaml)
        
    _optimize_asyncapi_file(spec_path)

[INFO] __main__: Executing 'asyncapi optimize' on the generated asyncapi specification file:
[INFO] __main__: No optimization has been applied since /tmp/tmp7j5z3v9g/asyncapi.yml looks optimized!



In [ ]:
valid_yaml2 = """
asyncapi: 2.0.0
info:
  title: Streetlights API
  version: '1.0.0'
channels:
  smartylighting/event/{streetlightId}/lighting/measured:
    parameters:
      #this parameter is duplicated. it can be moved to components and ref-ed from here.
      streetlightId:
        schema:
          type: string
    subscribe:
      operationId: receiveLightMeasurement
      traits:
        - bindings:
            kafka:
              clientId: my-app-id
      message:
        name: lightMeasured
        title: Light measured
        contentType: application/json
        traits:
          - headers:
              type: object
              properties:
                my-app-header:
                  type: integer
                  minimum: 0
                  maximum: 100
        payload:
          type: object
          properties:
            lumens:
              type: integer
              minimum: 0
            #full form is used, we can ref it to: #/components/schemas/sentAt
            sentAt:
              type: string
              format: date-time
  smartylighting/action/{streetlightId}/turn/on:
    parameters:
      streetlightId:
        schema:
          type: string
    publish:
      operationId: turnOn
      traits:
        - bindings:
            kafka:
              clientId: my-app-id
      message:
        name: turnOnOff
        title: Turn on/off
        traits:
          - headers:
              type: object
              properties:
                my-app-header:
                  type: integer
                  minimum: 0
                  maximum: 100
        payload:
          type: object
          properties:
            sentAt:
              $ref: "#/components/schemas/sentAt"
components:
  messages:
    #libarary should be able to find and delete this message, because it is not used anywhere.
    unusedMessage:
      name: unusedMessage
      title: This message is not used in any channel.
      
  schemas:
    #this schema is ref-ed in one channel and used full form in another. library should be able to identify and ref the second channel as well.
    sentAt:
      type: string
      format: date-time
"""

with TemporaryDirectory() as d:
    spec_path = Path(d) / ASYNC_API_SPEC_FILE_NAME
    with open(spec_path, "w", encoding="utf-8") as f:
        f.write(valid_yaml2)
        
    _optimize_asyncapi_file(spec_path)

[INFO] __main__: Executing 'asyncapi optimize' on the generated asyncapi specification file:
[INFO] __main__: Created file /tmp/tmpv2a8gqtb/asyncapi_optimized.yml...



In [ ]:
try:
    raise ValueError(f"Maximum number of retries")
except ValueError as e:
    if "Maximum number of retries" in str(e):
        print(str(e))
    else:
        raise e


Maximum number of retries


In [ ]:
# | export

def _generate_asyncapi_spec(description: str, total_usage: List[Dict[str, int]]) -> Tuple[str, List[Dict[str, int]]]:
    async_spec_generator = CustomAIChat(user_prompt=ASYNCAPI_SPEC_GENERATION_PROMPT)
    async_spec_validator = ValidateAndFixResponse(async_spec_generator, _validate_response, max_attempts=3)
    validated_async_spec, total_usage = async_spec_validator.fix(description, total_usage)
    return validated_async_spec, total_usage

def generate_asyncapi_spec(description: str, output_path: str, total_usage: List[Dict[str, int]], max_attempts: int = MAX_ASYNC_SPEC_RETRIES) -> List[Dict[str, int]]:
    """Generate a AsyncAPI spec from the user's application description

    Args:
        description: Validated User application description
        output_path: The path to the output file where the generated AsyncAPI spec will be saved.
        max_attempts: An optional integer specifying the maximum number of attempts to generate asyncapi specification (number of fixes are not included in this number).

    Returns:
        Appends total token used to generate the AsyncAPI spec to the end of total_usage list
    """
    with yaspin(
        text="Generating AsyncAPI specification (usually takes around 15 to 30 seconds)...",
        color="cyan",
        spinner="clock",
    ) as sp:
        iterations: int = 0
        while True:
            logger.info(f"\nGenerating AsyncAPI specification - {iterations + 1}. attempt")      
            try:
                validated_async_spec, total_usage = _generate_asyncapi_spec(description, total_usage)
                break
            except ValueError as e:
                if MAX_NUM_FIXES_MSG not in str(e) or iterations >= max_attempts:
                    raise e
                # Try to generate specifiction from the beginning
                iterations += 1

        output_file = f"{output_path}/{ASYNC_API_SPEC_FILE_NAME}"
        write_file_contents(output_file, validated_async_spec)
        _asyncapi_to_latest_version(output_file)
        _optimize_asyncapi_file(output_file)

        sp.text = ""
        sp.ok(f" ✔ AsyncAPI specification generated and saved to: {output_file}")
        return total_usage

In [ ]:
# | notest


app_description = """
Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. 
Then, create a new object with the attributes country and store_product, and set the country to "IND" and the store_product to message. Finally, publish the consumed message to 'change_currency' topic.

Use SASL_SSL with SCRAM-SHA-512 for authentication with username and password.
"""


with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{ASYNC_API_SPEC_FILE_NAME}"
    
    total_usage = generate_asyncapi_spec(app_description, output_path, [])
    
    assert Path(output_path).exists()
    actual = [file for file in Path(output_path).iterdir()]
    assert str(actual[0]) == output_file
    
    with open(output_file, 'r', encoding="utf-8") as f:
        yaml_data = f.read()
    print(yaml_data)

assert int(total_usage[0]["total_tokens"]) > 0
print(total_usage)

[INFO] __main__: 
Generating AsyncAPI specification - 1. attempt
⠹ Generating AsyncAPI specification (usually takes around 15 to 30 seconds)... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:120: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


⠏ Generating AsyncAPI specification (usually takes around 15 to 30 seconds)... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
asyncapi.yml:12:14 error asyncapi-document-resolved ""security" property type must be array"
asyncapi.yml:21:14 error asyncapi-document-resolved ""security" property type must be array"
asyncapi.yml:30:14 error asyncapi-document-resolved ""security" property type must be array"

Below is the updated prompt message along with the previously generated invalid response:

Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If i

KeyboardInterrupt: 

In [ ]:
# | notest

app_description = "Create a FastKafka application"

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/asyncapi.yml"
        
    with pytest.raises(ValueError) as e:
        total_usage = generate_asyncapi_spec(app_description, output_path, [])
    print(e.value)
        
    assert "==== INCOMPLETE APP DESCRIPTION ====" in str(e.value)

⠋ Generating AsyncAPI specification (usually takes around 15 to 30 seconds)...[INFO] __main__: 
Generating AsyncAPI specification - 1. attempt
⠹ Generating AsyncAPI specification (usually takes around 15 to 30 seconds)... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:120: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


⠇ Generating AsyncAPI specification (usually takes around 15 to 30 seconds)... [INFO] faststream_gen._code_generator.helper: Validation failed due to the following errors, trying again...
asyncapi.yml:12:3 error asyncapi-document-resolved ""channels" property type must be object"
asyncapi.yml:15:5 error asyncapi-document-resolved ""messages" property type must be object"
asyncapi.yml:17:5 error asyncapi-document-resolved ""schemas" property type must be object"

Below is the updated prompt message along with the previously generated invalid response:
Create a FastKafka application

==== YOUR RESPONSE (WITH ISSUES) ====

asyncapi: 2.6.0
info:
  title: FastKafka Application
  version: 0.0.1
  description: "A FastKafka application for building asynchronous services that interact with Kafka topics. This application simplifies the process of writing producers and consumers for Kafka topics, handling parsing, networking, task scheduling, and data generation automatically. With FastKafka, dev

KeyboardInterrupt: 